# Word2Vec

In [1]:
# !pip install gensim

In [2]:
import gensim

print(gensim.__version__)

4.3.3


In [3]:
# !pip install nltk==3.8.1

## 영어 데이터 다운로드 및 전처리

In [4]:
import re
from lxml import etree
import urllib.request
import zipfile
import nltk
from nltk.tokenize import word_tokenize, sent_tokenize

In [10]:
nltk.download('punkt')
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [6]:
urllib.request.urlretrieve("https://raw.githubusercontent.com/GaoleMeng/RNN-and-FFNN-textClassification/master/ted_en-20160408.xml", filename="ted_en-20160408.xml")

('ted_en-20160408.xml', <http.client.HTTPMessage at 0x7aea0911a690>)

In [7]:
targetXML = open('ted_en-20160408.xml', 'r', encoding='UTF8')
target_text = etree.parse(targetXML)

# xml 파일로부터 <content>와 </content> 사이의 내용만 가져옴
parse_text = '\n'.join(target_text.xpath('//content/text()'))

# 정규 표현식의 sub 모듈을 통해 content 중간에 등장하는 (Audio), (Laughter) 등의 배경음 부분을 제거.
content_text = re.sub(r'\([^)]*\)', '', parse_text)

In [8]:
len(content_text)

24062319

In [11]:
sent_text = sent_tokenize(content_text)

In [12]:
# 각 문장에 대해서 구두점을 제거 대문자를 소문자로 변환
normalized_text = []
for string in sent_text:
     tokens = re.sub(r"[^a-z0-9]+", " ", string.lower())
     normalized_text.append(tokens)

result = [word_tokenize(sentence) for sentence in normalized_text]

In [13]:
print('총 샘플의 개수 : {}'.format(len(result)))

총 샘플의 개수 : 273424


In [14]:
for line in result[:3]: # 샘플 3개만 출력
    print(line)

['here', 'are', 'two', 'reasons', 'companies', 'fail', 'they', 'only', 'do', 'more', 'of', 'the', 'same', 'or', 'they', 'only', 'do', 'what', 's', 'new']
['to', 'me', 'the', 'real', 'real', 'solution', 'to', 'quality', 'growth', 'is', 'figuring', 'out', 'the', 'balance', 'between', 'two', 'activities', 'exploration', 'and', 'exploitation']
['both', 'are', 'necessary', 'but', 'it', 'can', 'be', 'too', 'much', 'of', 'a', 'good', 'thing']


## 영어 Word2Vec 훈련시키기

In [15]:
from gensim.models import Word2Vec

model = Word2Vec(sentences=result, vector_size=100, window=5, min_count=5, workers=4, sg=0)

Word2Vec의 하이퍼파라미터

vector_size = 워드 벡터의 특징 값. 즉, 임베딩 된 벡터의 차원  
window = 컨텍스트 윈도우 크기  
min_count = 단어 최소 빈도 수 제한 (빈도가 적은 단어들은 학습하지 않음)  
workers = 학습을 위한 프로세스 수  
sg = 0은 CBOW, 1은 Skip-gram.  

코사인 유사도라는 것을 유사도 메트릭으로 사용. 값의 범위가 -1 ~ 1

In [16]:
model_result = model.wv.most_similar("man")
print(model_result)

[('woman', 0.8572914600372314), ('guy', 0.803706705570221), ('lady', 0.7722309231758118), ('boy', 0.755868673324585), ('girl', 0.7381670475006104), ('gentleman', 0.717999279499054), ('soldier', 0.7059162855148315), ('kid', 0.6765409708023071), ('friend', 0.643997848033905), ('king', 0.6364792585372925)]


In [17]:
model.wv["man"]

array([-0.11592601, -2.1754398 , -0.21555255, -0.08241018,  1.21284   ,
       -0.04503538,  0.7800172 ,  0.10151172,  0.24364293, -0.6124928 ,
       -1.5779419 , -0.42522722, -0.10586189,  0.09919971, -0.6655095 ,
       -0.42871243,  0.25625333, -0.24471807,  0.85824245, -0.6877128 ,
       -0.45461828,  1.6219492 , -0.6726265 , -0.8015764 ,  1.7042222 ,
        0.4472823 , -1.9898179 , -1.1128294 , -0.36575103, -0.9926194 ,
        0.4659659 ,  0.64966834,  0.98435265,  0.45224452, -1.167058  ,
       -1.0662541 , -0.40361437, -1.0416789 , -1.9432207 , -0.9591105 ,
        0.56623465, -2.0901747 ,  1.0149791 ,  1.458879  ,  0.3842855 ,
       -1.1922266 , -1.3889843 , -1.8105863 , -0.9149323 , -0.36515155,
       -0.42152864, -1.8930278 ,  0.15438029,  1.4553442 , -0.28134695,
        0.34374842,  0.338683  , -0.77642936, -1.2544035 , -1.1508238 ,
       -0.77621037,  0.19944115,  0.46355692,  1.838779  , -1.8441784 ,
       -0.12403382, -0.25515547, -0.20538276, -1.6445787 ,  0.99

In [18]:
len(model.wv["man"])

100

In [19]:
from gensim.models import KeyedVectors

model.wv.save_word2vec_format('eng_w2v')
loaded_model = KeyedVectors.load_word2vec_format("eng_w2v")

In [20]:
model_result = loaded_model.most_similar("man")
print(model_result)

[('woman', 0.8572914600372314), ('guy', 0.803706705570221), ('lady', 0.7722309231758118), ('boy', 0.755868673324585), ('girl', 0.7381670475006104), ('gentleman', 0.717999279499054), ('soldier', 0.7059162855148315), ('kid', 0.6765409708023071), ('friend', 0.643997848033905), ('king', 0.6364792585372925)]


In [21]:
model.wv.vectors.shape

(21613, 100)

In [22]:
# 현재 경로
%pwd

'/content'

## 한국어 데이터 다운로드 및 전처리

In [24]:
# !pip install konlpy
# !pip install mecab-python
# !bash <(curl -s https://raw.githubusercontent.com/konlpy/konlpy/master/scripts/mecab.sh)

In [25]:
import urllib.request
from konlpy.tag import Mecab
from gensim.models.word2vec import Word2Vec
import pandas as pd
import matplotlib.pyplot as plt

In [26]:
urllib.request.urlretrieve("https://raw.githubusercontent.com/e9t/nsmc/master/ratings.txt", filename="ratings.txt")

('ratings.txt', <http.client.HTTPMessage at 0x7ae9ea8477d0>)

In [27]:
train_data = pd.read_table('ratings.txt')

In [28]:
train_data[:5] # 상위 5개 출력

,id,document,label
0,8112052,어릴때보고 지금다시봐도 재밌어요ㅋㅋ,1
1,8132799,"디자인을 배우는 학생으로, 외국디자이너와 그들이 일군 전통을 통해 발전해가는 문화산...",1
2,4655635,폴리스스토리 시리즈는 1부터 뉴까지 버릴께 하나도 없음.. 최고.,1
3,9251303,와.. 연기가 진짜 개쩔구나.. 지루할거라고 생각했는데 몰입해서 봤다.. 그래 이런...,1
4,10067386,안개 자욱한 밤하늘에 떠 있는 초승달 같은 영화.,1


In [29]:
print(len(train_data)) # 리뷰 개수 출력

200000


In [30]:
# NULL 값 존재 유무
print(train_data.isnull().values.any())

True


In [32]:
train_data = train_data.dropna(how = 'any')
print(train_data.isnull().values.any())

False


In [33]:
print(len(train_data))

199992


In [34]:
# 정규 표현식을 통한 한글 외 제거
train_data['document'] = train_data['document'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")

In [35]:
train_data[:5] # 상위 5개 출력

,id,document,label
0,8112052,어릴때보고 지금다시봐도 재밌어요ㅋㅋ,1
1,8132799,"디자인을 배우는 학생으로, 외국디자이너와 그들이 일군 전통을 통해 발전해가는 문화산...",1
2,4655635,폴리스스토리 시리즈는 1부터 뉴까지 버릴께 하나도 없음.. 최고.,1
3,9251303,와.. 연기가 진짜 개쩔구나.. 지루할거라고 생각했는데 몰입해서 봤다.. 그래 이런...,1
4,10067386,안개 자욱한 밤하늘에 떠 있는 초승달 같은 영화.,1


In [36]:
# 불용어 정의
stopwords = ['도', '는', '다', '의', '가', '이', '은', '한', '에', '하', '고', '을', '를', '인', '듯', '과', '와', '네', '들', '듯', '지', '임', '게']

In [37]:
mecab = Mecab()

tokenized_data = []
for sentence in train_data['document']:
    temp_X = mecab.morphs(sentence) # 토큰화
    temp_X = [word for word in temp_X if not word in stopwords] # 불용어 제거
    tokenized_data.append(temp_X)

In [38]:
print(tokenized_data[:3])

[['어릴', '때', '보', '지금', '다시', '봐도', '재밌', '어요', 'ㅋㅋ'], ['디자인', '배우', '학생', '으로', ',', '외국', '디자이너', '그', '일군', '전통', '통해', '발전', '해', '문화', '산업', '부러웠', '는데', '.', '사실', '우리', '나라', '에서', '그', '어려운', '시절', '끝', '까지', '열정', '지킨', '노라노', '같', '전통', '있', '어', '저', '같', '사람', '꿈', '꾸', '이뤄나갈', '수', '있', '다는', '것', '감사', '합니다', '.'], ['폴리스', '스토리', '시리즈', '1', '부터', '뉴', '까지', '버릴', '께', '하나', '없', '음', '.', '.', '최고', '.']]


## 한국어 Word2Vec 훈련시키기

In [40]:
from gensim.models import Word2Vec

model = Word2Vec(sentences = tokenized_data, vector_size = 100, window = 5, min_count = 5, workers = 4, sg = 0)

In [41]:
model.wv.vectors.shape

(18939, 100)

In [42]:
print(model.wv.most_similar("최민식"))

[('한석규', 0.8667585849761963), ('드니로', 0.8481647372245789), ('안성기', 0.8445639610290527), ('송강호', 0.8345094323158264), ('주진모', 0.8265935778617859), ('최민수', 0.8120617866516113), ('박중훈', 0.8020917177200317), ('정재영', 0.8008133172988892), ('설경구', 0.7990785241127014), ('박신양', 0.7990480065345764)]


In [43]:
model.wv['최민식']

array([-0.31764644,  0.5323253 , -0.23170447,  0.09802543, -0.03901153,
       -0.21731746,  0.2777344 ,  0.52935714, -0.08887305, -0.2802077 ,
        0.0023508 , -0.34063518,  0.00458707, -0.02598609,  0.0506565 ,
       -0.4260378 ,  0.21271919, -0.10221633,  0.17952755, -0.26704964,
       -0.0837279 , -0.35273874,  0.20784912,  0.24395782,  0.05977842,
       -0.23726992,  0.30810106, -0.16312435, -0.37035978, -0.00831317,
        0.6295186 , -0.0751156 ,  0.7227886 , -0.08674344,  0.27975246,
        0.32159814,  0.05546604, -0.16309924, -0.2432814 , -0.34091088,
        0.2330927 , -0.2206438 , -0.1097967 ,  0.1316833 ,  0.060969  ,
       -0.08336803, -0.17266421,  0.21838214, -0.03149402,  0.0838527 ,
        0.16859703, -0.13940531, -0.22213914,  0.00268317, -0.29734898,
       -0.03894559,  0.22956672,  0.00253938,  0.17710826, -0.29798186,
        0.13231352,  0.04569581,  0.02456356,  0.03931379, -0.10147397,
        0.04879308,  0.24592197,  0.17705764, -0.30385318,  0.09

In [44]:
print(model.wv.most_similar("히어로"))

[('호러', 0.8619926571846008), ('슬래셔', 0.8458645939826965), ('블록버스터', 0.8347368836402893), ('SF', 0.8181436657905579), ('정통', 0.8061800003051758), ('패러디', 0.805067241191864), ('하이틴', 0.7975485324859619), ('sf', 0.793605625629425), ('무비', 0.7884343862533569), ('무협', 0.7847630381584167)]


In [45]:
# 영어 모델이 저장된 경로로 이동
%cd /content

/content


In [47]:
from gensim.models import KeyedVectors

model.wv.save_word2vec_format('kor_w2v')

## 영어 Word2Vec 시각화

In [48]:
!python -m gensim.scripts.word2vec2tensor --input eng_w2v --output eng_w2v

2025-04-08 07:43:27,698 - word2vec2tensor - INFO - running /usr/local/lib/python3.11/dist-packages/gensim/scripts/word2vec2tensor.py --input eng_w2v --output eng_w2v
2025-04-08 07:43:27,698 - keyedvectors - INFO - loading projection weights from eng_w2v
2025-04-08 07:43:29,531 - utils - INFO - KeyedVectors lifecycle event {'msg': 'loaded (21613, 100) matrix of type float32 from eng_w2v', 'binary': False, 'encoding': 'utf8', 'datetime': '2025-04-08T07:43:29.530180', 'gensim': '4.3.3', 'python': '3.11.11 (main, Dec  4 2024, 08:55:07) [GCC 11.4.0]', 'platform': 'Linux-6.1.85+-x86_64-with-glibc2.35', 'event': 'load_word2vec_format'}
2025-04-08 07:43:30,716 - word2vec2tensor - INFO - 2D tensor file saved to eng_w2v_tensor.tsv
2025-04-08 07:43:30,716 - word2vec2tensor - INFO - Tensor metadata file saved to eng_w2v_metadata.tsv
2025-04-08 07:43:30,717 - word2vec2tensor - INFO - finished running word2vec2tensor.py


https://projector.tensorflow.org/  



## 한국어 Word2Vec 시각화하기

In [49]:
!python -m gensim.scripts.word2vec2tensor --input kor_w2v --output kor_w2v

2025-04-08 07:43:58,631 - word2vec2tensor - INFO - running /usr/local/lib/python3.11/dist-packages/gensim/scripts/word2vec2tensor.py --input kor_w2v --output kor_w2v
2025-04-08 07:43:58,631 - keyedvectors - INFO - loading projection weights from kor_w2v
2025-04-08 07:43:59,730 - utils - INFO - KeyedVectors lifecycle event {'msg': 'loaded (18939, 100) matrix of type float32 from kor_w2v', 'binary': False, 'encoding': 'utf8', 'datetime': '2025-04-08T07:43:59.729163', 'gensim': '4.3.3', 'python': '3.11.11 (main, Dec  4 2024, 08:55:07) [GCC 11.4.0]', 'platform': 'Linux-6.1.85+-x86_64-with-glibc2.35', 'event': 'load_word2vec_format'}
2025-04-08 07:44:00,789 - word2vec2tensor - INFO - 2D tensor file saved to kor_w2v_tensor.tsv
2025-04-08 07:44:00,789 - word2vec2tensor - INFO - Tensor metadata file saved to kor_w2v_metadata.tsv
2025-04-08 07:44:00,790 - word2vec2tensor - INFO - finished running word2vec2tensor.py


# FastText

## Word2Vec의 OOV 문제 확인해보기

OOV 문제(Out-Of-Vocabulary Problem) : Vocabulary에 존재하지 않는 단어가 등장하는 문제

In [50]:
loaded_model = KeyedVectors.load_word2vec_format("eng_w2v")

In [51]:
model_result = loaded_model.most_similar("overacting")
print(model_result)

KeyError: "Key 'overacting' not present in vocabulary"

In [52]:
model_result = loaded_model.most_similar("memory")
print(model_result)

[('perception', 0.6799529194831848), ('vision', 0.679359495639801), ('brain', 0.6761564612388611), ('consciousness', 0.6715499758720398), ('body', 0.6635928750038147), ('gut', 0.6537912487983704), ('tissue', 0.6451292037963867), ('imagination', 0.638461172580719), ('essence', 0.6370823979377747), ('capability', 0.631279706954956)]


In [53]:
model_result = loaded_model.most_similar("memorry")
print(model_result)

KeyError: "Key 'memorry' not present in vocabulary"

In [54]:
model_result = loaded_model.most_similar("electrofishing")
print(model_result)

KeyError: "Key 'electrofishing' not present in vocabulary"

## FastText로 같은 단어에 대해서 테스트해보기

In [55]:
from gensim.models import FastText

fasttext_model = FastText(result, vector_size=100, window=5, min_count=5, workers=4, sg=1)

In [56]:
fasttext_model.wv.most_similar('overacting')

[('distracting', 0.8835641741752625),
 ('subtracting', 0.8803499341011047),
 ('contracting', 0.8748215436935425),
 ('interacting', 0.8671647906303406),
 ('manipulating', 0.861724317073822),
 ('impacting', 0.8491047024726868),
 ('overarching', 0.8473291397094727),
 ('dissecting', 0.843493640422821),
 ('acting', 0.8397181034088135),
 ('regurgitating', 0.8375457525253296)]

In [57]:
fasttext_model.wv.most_similar('memorry')

[('memo', 0.8161763548851013),
 ('memoir', 0.783028244972229),
 ('forgery', 0.7721934914588928),
 ('rehearse', 0.7622562646865845),
 ('nemo', 0.7621496915817261),
 ('brochure', 0.758842945098877),
 ('memorize', 0.7555573582649231),
 ('rehearsal', 0.7487052083015442),
 ('memory', 0.7467703223228455),
 ('klee', 0.7370795607566833)]

In [58]:
fasttext_model.wv.most_similar("electrofishing")

[('electrolux', 0.8645876049995422),
 ('electrolyte', 0.8626345992088318),
 ('electro', 0.8560693264007568),
 ('electroshock', 0.8536320328712463),
 ('electroencephalogram', 0.8494594693183899),
 ('electrogram', 0.8356858491897583),
 ('electric', 0.8286804556846619),
 ('airbus', 0.8257551789283752),
 ('electronic', 0.8238762617111206),
 ('electron', 0.8232104182243347)]

# 자모 단위 FastText

In [60]:
# 한글 자모 단위 처리 패키지 설치

# !pip install hgtk

In [62]:
# fasttext 설치

# !git clone https://github.com/facebookresearch/fastText.git
# %cd fastText
# !make
# !pip install .

## hgtk 튜토리얼

In [63]:
import hgtk

In [64]:
urllib.request.urlretrieve("https://raw.githubusercontent.com/bab2min/corpus/master/sentiment/naver_shopping.txt", filename="ratings_total.txt")

('ratings_total.txt', <http.client.HTTPMessage at 0x7ae9e8491c90>)

In [65]:
total_data = pd.read_table('ratings_total.txt', names=['ratings', 'reviews'])
print('전체 리뷰 개수 :',len(total_data))

전체 리뷰 개수 : 200000


In [66]:
total_data[:5]

,ratings,reviews
0,5,배공빠르고 굿
1,2,택배가 엉망이네용 저희집 밑에층에 말도없이 놔두고가고
2,5,아주좋아요 바지 정말 좋아서2개 더 구매했어요 이가격에 대박입니다. 바느질이 조금 ...
3,2,선물용으로 빨리 받아서 전달했어야 하는 상품이었는데 머그컵만 와서 당황했습니다. 전...
4,5,민트색상 예뻐요. 옆 손잡이는 거는 용도로도 사용되네요 ㅎㅎ


한글의 자모를 처리하는 패키지인 hgtk  
hgtk의 checker를 사용하면 입력이 한글인지 아닌지를 판단하여 True 또는 False를 리턴

In [67]:
# 한글인지 체크
hgtk.checker.is_hangul('ㄱ')

True

In [68]:
# 한글인지 체크
hgtk.checker.is_hangul('28')

False

hgtk의 letter를 사용하면 음절을 자모 단위로 분리하거나, 자모의 시퀀스를 다시 음절로 조합 가능  
이는 각각 decompose와 compose로 가능

In [69]:
# 음절을 초성, 중성, 종성으로 분해
hgtk.letter.decompose('남')

('ㄴ', 'ㅏ', 'ㅁ')

In [70]:
# 초성, 중성을 결합
hgtk.letter.compose('ㄴ', 'ㅏ')

'나'

In [71]:
# 초성, 중성, 종성을 결합
hgtk.letter.compose('ㄴ', 'ㅏ', 'ㅁ')

'남'

In [72]:
# 한글이 아닌 입력에 대해서는 에러 발생

hgtk.letter.decompose('1')

NotHangulException: 

In [73]:
# 결합할 수 없는 상황에서는 에러 발생

hgtk.letter.compose('ㄴ', 'ㅁ', 'ㅁ')

NotHangulException: No valid Hangul character index

## 데이터 전처리

In [74]:
def word_to_jamo(token):
  def to_special_token(jamo):
    if not jamo:
      return '-'
    else:
      return jamo

  decomposed_token = ''
  for char in token:
    try:
      # char(음절)을 초성, 중성, 종성으로 분리
      cho, jung, jong = hgtk.letter.decompose(char)

      # 자모가 빈 문자일 경우 특수문자 -로 대체
      cho = to_special_token(cho)
      jung = to_special_token(jung)
      jong = to_special_token(jong)
      decomposed_token = decomposed_token + cho + jung + jong

    # 만약 char(음절)이 한글이 아닐 경우 자모를 나누지 않고 추가
    except Exception as exception:
      if type(exception).__name__ == 'NotHangulException':
        decomposed_token += char

  # 단어 토큰의 자모 단위 분리 결과를 추가
  return decomposed_token

In [75]:
word_to_jamo('남동생')

'ㄴㅏㅁㄷㅗㅇㅅㅐㅇ'

In [76]:
word_to_jamo('여동생')

'ㅇㅕ-ㄷㅗㅇㅅㅐㅇ'

In [77]:
mecab = Mecab()

In [78]:
print(mecab.morphs('선물용으로 빨리 받아서 전달했어야 하는 상품이었는데 머그컵만 와서 당황했습니다.'))

['선물', '용', '으로', '빨리', '받', '아서', '전달', '했어야', '하', '는', '상품', '이', '었', '는데', '머그', '컵', '만', '와서', '당황', '했', '습니다', '.']


In [79]:
def tokenize_by_jamo(s):
    return [word_to_jamo(token) for token in mecab.morphs(s)]

In [80]:
# 형태소 분석 후 자모로 전환

print(tokenize_by_jamo('선물용으로 빨리 받아서 전달했어야 하는 상품이었는데 머그컵만 와서 당황했습니다.'))

['ㅅㅓㄴㅁㅜㄹ', 'ㅇㅛㅇ', 'ㅇㅡ-ㄹㅗ-', 'ㅃㅏㄹㄹㅣ-', 'ㅂㅏㄷ', 'ㅇㅏ-ㅅㅓ-', 'ㅈㅓㄴㄷㅏㄹ', 'ㅎㅐㅆㅇㅓ-ㅇㅑ-', 'ㅎㅏ-', 'ㄴㅡㄴ', 'ㅅㅏㅇㅍㅜㅁ', 'ㅇㅣ-', 'ㅇㅓㅆ', 'ㄴㅡㄴㄷㅔ-', 'ㅁㅓ-ㄱㅡ-', 'ㅋㅓㅂ', 'ㅁㅏㄴ', 'ㅇㅘ-ㅅㅓ-', 'ㄷㅏㅇㅎㅘㅇ', 'ㅎㅐㅆ', 'ㅅㅡㅂㄴㅣ-ㄷㅏ-', '.']


자모 단위 FastText에서는 각 형태소 분석 결과 토큰들이 추가적으로 자모 단위로 분해된 토큰들을 가지고 학습을 진행함


In [81]:
from tqdm import tqdm

In [82]:
tokenized_data = []

for sample in total_data['reviews'].to_list():
    tokenzied_sample = tokenize_by_jamo(sample) # 자소 단위 토큰화
    tokenized_data.append(tokenzied_sample)

In [83]:
tokenized_data[0]

['ㅂㅐ-ㄱㅗㅇ', 'ㅃㅏ-ㄹㅡ-', 'ㄱㅗ-', 'ㄱㅜㅅ']

기존의 단어가 무엇이었는지를 알아보기 위해 초성, 중성, 종성을 입력받으면 역으로 단어로 바꿔주는 jamo_to_word 함수를 구현

In [84]:
def jamo_to_word(jamo_sequence):
  tokenized_jamo = []
  index = 0

# 자모단위 토큰화
  while index < len(jamo_sequence):
    # 문자가 한글(정상적인 자모)이 아닐 경우
    if not hgtk.checker.is_hangul(jamo_sequence[index]):
      tokenized_jamo.append(jamo_sequence[index])
      index = index + 1

    # 문자가 정상적인 자모라면 초성, 중성, 종성을 하나의 토큰으로 간주.
    else:
      tokenized_jamo.append(jamo_sequence[index:index + 3])
      index = index + 3

  word = ''
  try:
    for jamo in tokenized_jamo:

      # 초성, 중성, 종성의 묶음으로 추정되는 경우
      if len(jamo) == 3:
        if jamo[2] == "-":
          # 종성이 존재하지 않는 경우
          word = word + hgtk.letter.compose(jamo[0], jamo[1])
        else:
          # 종성이 존재하는 경우
          word = word + hgtk.letter.compose(jamo[0], jamo[1], jamo[2])
      # 한글이 아닌 경우
      else:
        word = word + jamo

  # 복원 중(hgtk.letter.compose) 에러 발생 시 초기 입력 리턴
  except Exception as exception:
    if type(exception).__name__ == 'NotHangulException':
      return jamo_sequence

  return word

In [85]:
jamo_to_word('ㄴㅏㅁㄷㅗㅇㅅㅐㅇ')

'남동생'

## FastText

In [86]:
import fasttext

In [87]:
with open('tokenized_data.txt', 'w') as out:
  for line in tqdm(tokenized_data, unit=' line'):
    out.write(' '.join(line) + '\n')

100%|██████████| 200000/200000 [00:00<00:00, 485596.86 line/s]


In [88]:
model = fasttext.train_unsupervised('tokenized_data.txt', model='cbow')

In [89]:
model.save_model("fasttext.bin")

In [90]:
model = fasttext.load_model("fasttext.bin")

In [ ]:
model[word_to_jamo('남동생')]

array([-0.75121707, -0.09777343, -0.06479282,  0.3878322 , -0.3629536 ,
        0.444919  ,  0.17368399, -0.6492414 , -0.1460425 , -0.6556199 ,
        0.47561774,  0.15500426, -0.07597088,  0.6838326 , -0.12038054,
       -0.05893848,  0.80528766,  0.41876066,  0.5232016 , -0.76272345,
       -0.38435844, -0.48213163,  0.47253323,  0.42145985, -0.40317082,
        0.1875647 ,  0.40562132,  0.0864673 ,  0.4318442 ,  0.41785347,
        0.99820423, -1.1779842 ,  0.02945196,  0.33049268,  1.0954536 ,
        0.03768162,  0.44268563, -1.095168  , -0.23283842,  0.5355596 ,
        0.6794727 ,  0.47900698, -0.60658246,  1.2562438 , -0.24471879,
        0.46079734, -0.01676963, -0.17489414, -0.43604985,  0.34543806,
       -0.15913598,  0.2918002 ,  0.8731358 ,  1.0437014 , -0.6025581 ,
        0.2981697 ,  0.53655666, -0.05084229,  0.05335928,  0.2421371 ,
        0.17908356, -0.6660127 ,  0.934657  ,  0.09646372, -0.6387341 ,
       -0.25673625, -0.77488786, -0.07015391, -0.37226695, -0.55

남동생 '벡터'와 가장 유사도가 높은 벡터  
 get_nearest_neighbors()

In [91]:
model.get_nearest_neighbors(word_to_jamo('남동생'), k=10)

[(0.8671373724937439, 'ㄷㅗㅇㅅㅐㅇ'),
 (0.8345811367034912, 'ㄴㅏㅁㅊㅣㄴ'),
 (0.7394193410873413, 'ㄴㅏㅁㅍㅕㄴ'),
 (0.7316157817840576, 'ㅊㅣㄴㄱㅜ-'),
 (0.7173355221748352, 'ㅅㅐㅇㅇㅣㄹ'),
 (0.7168329358100891, 'ㄴㅏㅁㅇㅏ-'),
 (0.7005258202552795, 'ㅈㅗ-ㅋㅏ-'),
 (0.6888477802276611, 'ㅈㅜㅇㅎㅏㄱㅅㅐㅇ'),
 (0.6667895317077637, 'ㅇㅓㄴㄴㅣ-'),
 (0.6643229126930237, 'ㄴㅏㅁㅈㅏ-')]

In [92]:
def transform(word_sequence):
  return [(jamo_to_word(word), similarity) for (similarity, word) in word_sequence]

In [93]:
print(transform(model.get_nearest_neighbors(word_to_jamo('남동생'), k=10)))

[('동생', 0.8671373724937439), ('남친', 0.8345811367034912), ('남편', 0.7394193410873413), ('친구', 0.7316157817840576), ('생일', 0.7173355221748352), ('남아', 0.7168329358100891), ('조카', 0.7005258202552795), ('중학생', 0.6888477802276611), ('언니', 0.6667895317077637), ('남자', 0.6643229126930237)]


In [94]:
print(transform(model.get_nearest_neighbors(word_to_jamo('남동쉥'), k=10)))

[('남동생', 0.8909438252449036), ('남친', 0.8003354668617249), ('남매', 0.7774966955184937), ('남김', 0.7451346516609192), ('남긴', 0.7383974194526672), ('남짓', 0.7368336319923401), ('남녀', 0.7326962351799011), ('남아', 0.7286370992660522), ('남여', 0.7266424894332886), ('남길', 0.7219088077545166)]


In [95]:
print(transform(model.get_nearest_neighbors(word_to_jamo('남동셍ㅋ'), k=10)))

[('남동생', 0.8234370946884155), ('남친', 0.7265772819519043), ('남김', 0.7082480788230896), ('남길', 0.6784865260124207), ('남녀', 0.6686286330223083), ('남매', 0.6675403714179993), ('남여', 0.6633204817771912), ('남겼', 0.6621609926223755), ('남짓', 0.6599602103233337), ('남긴', 0.6571483016014099)]


In [96]:
print(transform(model.get_nearest_neighbors(word_to_jamo('난동생'), k=10)))

[('남동생', 0.8642392158508301), ('난생', 0.8244139552116394), ('남편', 0.8014969229698181), ('남친', 0.7568559646606445), ('동생', 0.7568278312683105), ('남아', 0.754828929901123), ('나눴', 0.7011327147483826), ('중학생', 0.7001649737358093), ('남자', 0.6799314022064209), ('신랑', 0.6761581897735596)]


In [97]:
print(transform(model.get_nearest_neighbors(word_to_jamo('낫동생'), k=10)))

[('남동생', 0.9303204417228699), ('동생', 0.8740969300270081), ('남편', 0.7611657381057739), ('남친', 0.7513895034790039), ('친구', 0.7390786409378052), ('중학생', 0.7209896445274353), ('조카', 0.7082139253616333), ('남아', 0.7011557817459106), ('난생', 0.7001751661300659), ('나눴', 0.6832748055458069)]


In [98]:
print(transform(model.get_nearest_neighbors(word_to_jamo('납동생'), k=10)))

[('남동생', 0.9049338102340698), ('동생', 0.8326806426048279), ('남편', 0.7896609902381897), ('남친', 0.7583615183830261), ('난생', 0.7417805790901184), ('중학생', 0.7253825664520264), ('남아', 0.7192257046699524), ('친구', 0.7001274824142456), ('나눴', 0.697450578212738), ('고등학생', 0.694034218788147)]


In [99]:
print(transform(model.get_nearest_neighbors(word_to_jamo('냚동생'), k=10)))

[('동생', 0.967219889163971), ('남동생', 0.8974405527114868), ('친구', 0.8116076588630676), ('조카', 0.7770885229110718), ('언니', 0.7635160088539124), ('딸', 0.7545560598373413), ('생일', 0.7490536570549011), ('딸애', 0.7439687252044678), ('중학생', 0.7377141714096069), ('남편', 0.7292447686195374)]


In [100]:
print(transform(model.get_nearest_neighbors(word_to_jamo('제품^^'), k=10)))

[('제품', 0.9371058344841003), ('제풍', 0.791787326335907), ('반제품', 0.7801757454872131), ('완제품', 0.7779927849769592), ('상품', 0.7686623930931091), ('타제품', 0.762442946434021), ('최상품', 0.7511221766471863), ('재품', 0.7040295004844666), ('화학제품', 0.695855438709259), ('중품', 0.6953531503677368)]
